# Obvious Open Mining Format (OOMF)
> "Work In Progress - Open Mining Format for fumans"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Mining, OMF, Seequent, Open Mining Format, Geostatistics]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true

The Open Mining Format (OMF) is a thoughtful proposal from the Global Mining Guidlines Group (GMG), suggesting a single data format for use in multiple mining software packages. This idea has been slow to gain support from some of the big players in this space. At this point the sole supporter of this concept (to my knowledge) is Seequent, the makers of Leapfrog3D software.  The use of multuiple commercial softwares or open source tools is becoming the norm for complex workflows focused on resource modeling. This necessiates the the ability to get data into and out-of commercial software packages. OMF is an attempt at this, unfortunately the oficial python tools for OMF are a bit clunky.


[Obvious Open Mining Format aka OOMF](https://github.com/ericbdaniels/Oomf) is an attempt to make working with the Open Mining Format more accessible by a more pythonic API. This is a work in progress and it should be noted that GMG already has OMF 2.0 in the works which will likely come with some breaking changes. The aim of OOMF is based on my own experience where a common workflow involves extracting information from and OMF file and manipulating it using some GSLIB or Python based tools and saving the results back to an OMF file. All of this can be done using the [OMF package](https://omf.readthedocs.io/en/latest/index.html). A quick read of the docs may leave you wanting a simpler approach. 

## Aims of OOMF
1. Provide a summary of OMF file contents.
2. Allow user access data saved within the OMF file by name.
3. Export to other common formats (CSV, VTK, GSLIB)
4. Easily create OMF files from existing data in CSV, VTK or GLIB formats.
5. Provide basic tools for visualization.

> The examples below cover points 1 through 3. 4 and 5 need a bit more work when time allows.


## Example usage
To demonstrate the purpose and use of OMF - consider the basic workflow of loading and exisiting OMF file, summarizing the contents, and exporting to another file format.

### Standard OMF approach

In [54]:
import omf
import pandas as pd

# First read the file 
reader = omf.OMFReader('./data/test_file.omf')
project = reader.get_project()

# Now see whats in the file - for each element print the name, some basic info and the names of all associated data attributes
for e in project.elements:
    print(f'Element Name: {e.name}, Origin: {e.geometry.origin}, Size: {e.geometry.num_cells}')
    for d in e.data:
        print(f'\t Attribute Name: {d.name}')

Element Name: collar, Origin: [0. 0. 0.], Size: 55
	 Attribute Name: holeid
Element Name: wolfpass_WP_assay, Origin: [0. 0. 0.], Size: 8583
	 Attribute Name: CU_pct
	 Attribute Name: DENSITY
	 Attribute Name: AG_gpt
	 Attribute Name: AU_gpt
	 Attribute Name: RECOV
	 Attribute Name: S_pct
	 Attribute Name: MO_ppm
	 Attribute Name: AS_ppm
	 Attribute Name: Final_cut
	 Attribute Name: holeid
Element Name: Topography, Origin: [0. 0. 0.], Size: 65936
	 Attribute Name: Elevation
Element Name: Basement, Origin: [0. 0. 0.], Size: 87702
Element Name: Early Diorite, Origin: [0. 0. 0.], Size: 79046
Element Name: Intermineral diorite, Origin: [0. 0. 0.], Size: 112216
Element Name: Dacite, Origin: [0. 0. 0.], Size: 39768
Element Name: Cover, Origin: [0. 0. 0.], Size: 102904
Element Name: Block Model, Origin: [444700. 492800.   2330.], Size: 1689600
	 Attribute Name: CU_pct


In [57]:
# Now access the attributes on the wolfpass_WP_assay element
e = project.elements[1]

#now get the data itself
assay_df = pd.DataFrame({d.name:d.array[:] for d in e.data})

assay_df.head()

,CU_pct,DENSITY,AG_gpt,AU_gpt,RECOV,S_pct,MO_ppm,AS_ppm,Final_cut,holeid
0,0.79,NaN,6.35,1.75,0.5,-0.01,10.0,26.3,2,0
1,0.83,NaN,5.20,1.73,1.3,-0.01,12.2,31.0,2,0
2,0.84,NaN,5.75,6.00,1.8,-0.01,24.8,32.5,2,0
3,0.83,2.32,2.85,2.56,1.8,0.20,15.7,13.9,2,0
4,0.97,2.98,2.90,1.53,2.0,0.50,14.8,15.5,2,0


> The OMF package works just fine, open the file get the data etc - no problem. Its just not inutitive because the user must access each element and attribute by its index in as list (or by its UUID which is even less user friendly in my opinion).

### OOMF approach

In [2]:
import oomf 

# read the file and summarize contents
prj = oomf.elements.Project("./data/test_file.omf")
prj.summarize()

In [6]:
# now access the assay data
assay_df = prj.elements.wolfpass_WP_assay.to_pandas()
assay_df.head()

,x,y,z,CU_pct,DENSITY,AG_gpt,AU_gpt,RECOV,S_pct,MO_ppm,AS_ppm,Final_cut,holeid
0,445198.219000,494110.594000,3057.757537,0.79,NaN,6.35,1.75,0.5,-0.01,10.0,26.3,2,0
1,445199.504526,494110.582781,3056.225448,0.83,NaN,5.20,1.73,1.3,-0.01,12.2,31.0,2,0
2,445200.790053,494110.571563,3054.693359,0.84,NaN,5.75,6.00,1.8,-0.01,24.8,32.5,2,0
3,445202.075579,494110.560344,3053.161270,0.83,2.32,2.85,2.56,1.8,0.20,15.7,13.9,2,0
4,445203.361105,494110.549126,3051.629181,0.97,2.98,2.90,1.53,2.0,0.50,14.8,15.5,2,0


* The nice summary above comes from `rich` which is an fantastic python package which allows for adding colored text, tables, summaries to any text output.
* Notice that any element is accessible by name, and any attribute associated with that object is also accessed by name: `prj.elements.wolfpass_WP_assay.CU_pct`, and to get the data: `prj.elements.wolfpass_WP_assay.CU_pct[:]`
* additionally, the coordinates are added to the dataframe by default. If using the stand OMF package coordinates exist separate from attributes.


### Supported File Formats

At this point export functions for CSV, VTK and GSLIB file formats are provided in addition to the pandas integration.

In [8]:
# to gslib
prj.elements.wolfpass_WP_assay.to_gslib("./data/gslib_export.dat")

# to csv 
prj.elements.wolfpass_WP_assay.to_csv("./data/csv_export.csv")

# to vtk 
prj.elements.wolfpass_WP_assay.to_vtk("./data/vtk_export.vtk")

> NOTE: The `to_gslib` and `to_csv` exports rely on pandas. This can be problematic for large datasets where loading the entire dataset into a pandas dataframe can either be particularly slow or lead to memory issues.

## Conclusion

If you use OMF files as part of your resource modeling workflow, hopefully this little tool will make life a bit easier. Its not perfect, there are tests to add, features to include, code to refactor but for now its what I had time for. In the future, I'd like to include an easy way to convert a group of GSLIB files or CSVs and create and OMF file. Additionally a 3D viewer would be nice. In the meantime, I'd welcome any contributions, comments etc.